In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext ip_linewatcher

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
np.set_printoptions(suppress=True)
output_notebook()

Loading BokehJS ...

In [2]:
#Draw map

time: 923 µs


In [5]:
df = pd.read_csv('./data/fahrzeitensollist2017010820170114.csv')


time: 4.21 s


In [6]:
df[['halt_id_von', 'halt_id_nach']].head()

,halt_id_von,halt_id_nach
0,2251,1906
1,1306,1502
2,1502,2657
3,2228,2590
4,1528,2228


time: 65.2 ms


In [7]:
df['journey_time'] =df['ist_an_nach1']-df['ist_an_von']
FOUR_HOURS = 60*60*4
filtered = df[df['journey_time']< FOUR_HOURS]
mean_journey_times = filtered.groupby(['halt_id_von', 'halt_id_nach'])['journey_time'].mean()
mean_journey_times.head()

halt_id_von  halt_id_nach
373          373              78.487942
             588             110.858521
588          373             115.506024
             1377            142.593248
701          2060             93.316060
Name: journey_time, dtype: float64

time: 465 ms


In [9]:
from collections import defaultdict
from route_server import Action

from_to = defaultdict(list)
cost_dict = dict()
for (halt_id_von, halt_id_nach), cost in mean_journey_times.items():
    from_to[halt_id_von].append(halt_id_nach)
    cost_dict[(halt_id_von, halt_id_nach)] = cost
    
class ActionFactory:
    def __init__(self, path_dict, costs):
        """path_dict contains successors for a halt_id
        costs contains the journey time in seconds for a leg, keyed by (from_halt_id, to_halt_id)"""
        self.path_dict = path_dict
        self.costs = costs
        
    def actions(self, path):
        """Return all valid paths, and costs, for the from_halt_id"""
        result = []
        for target in self.path_dict[path.end]:
            action = Action(end=target, cost=self.costs[(path.end, target)])
            result.append(action)
        return result

Actions = ActionFactory(from_to,cost_dict).actions

time: 34.3 ms


In [10]:
stops = pd.read_csv('./data/haltestelle.csv')
stops[['halt_id', 'halt_lang']][stops['halt_id'].isin([588, 1308, 1535, 1851,2008])]

,halt_id,halt_lang
3,588,"Zürich Flughafen, Bahnhof"
22,1308,"Zürich, ETH/Universitätsspital"
74,1535,"Zürich, Albisriederplatz"
121,1851,"Zürich, Sihlcity"
150,2008,"Dübendorf, Flugplatz"


time: 79.8 ms


In [11]:
from route_server import Puzzle, get_coordinates
search = Puzzle(Actions).graph_search
airport, sihlcity = (588, 1535)
paths = search(airport, sihlcity)
stop_ids = [x[0] for x in paths.extract_path()]

import folium
map_osm = folium.Map(location=[47.58023, 8.522235],
           zoom_start=10, 
                     #min_lat=46, max_lat=47.5, min_lon=8., max_lon=9,
                     min_zoom=10)

start = stop_ids[0]
stop = stop_ids[-1]
marker_cluster = folium.MarkerCluster().add_to(map_osm)
long,lat = get_coordinates(start)
folium.CircleMarker(location=[long,lat],
                            radius=5,
                            fill_color='blue').add_to(marker_cluster)

stop = stop_ids[-1]
marker_cluster = folium.MarkerCluster().add_to(map_osm)
long,lat = get_coordinates(stop)
folium.CircleMarker(location=[long,lat],
                            radius=5,
                            fill_color='blue').add_to(marker_cluster)

for start_id, stop_id in zip(stop_ids, stop_ids[1:]):
    try:
        s_long, s_lag = get_coordinates(start_id)
        e_long, e_lag = get_coordinates(stop_id)
        folium.PolyLine([[s_long, s_lag ],[e_long, e_lag]],

                        color="red", weight=2.5, opacity=1).add_to(map_osm)
    except:
        pass
map_osm
    #lat, long = get_coordinates(stop_id)
    #print(lat)

time: 647 ms
